In [9]:
import cv2
import numpy as np
import pandas as pd
from PIL import ImageFont, ImageDraw, Image
from tqdm import tqdm
import os
# === 入出力ファイルパス ===
base_path = "G:\kumamushi\Kumamushi_color\Kumamushi_color"
video1_path = os.path.join(base_path, "mCherry_GCaMP_saturation_alert_orthogonal_view_20240902-161310kumamushi_raw-1-1_275_487.avi")  # 蛍光
video2_path = os.path.join(base_path, "rotated_90r_4l_flipped_1-3_273x286.avi")  # 明視野
output_path = os.path.join(base_path, "merged_output_newlayout_outline_centered.avi")
timestamp_csv_path = os.path.join(base_path,"timestamp_selected_kumamushi_v2.csv")  # ← 追加：CSVのパス

# === Arialフォントのパス（Windows）===
font_path = "C:/Windows/Fonts/arial.ttf"
font_size = 40
font = ImageFont.truetype(font_path, font_size)

def put_text_arial(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# === 黒背景のサイズ ===
canvas_width = 2300
canvas_height = 900

# === 読み込み ===
cap1 = cv2.VideoCapture(video1_path)
cap2 = cv2.VideoCapture(video2_path)
if not cap1.isOpened() or not cap2.isOpened():
    raise RuntimeError("❌ 動画の読み込みに失敗しました")

# === タイムスタンプCSV読み込み ===
df = pd.read_csv(timestamp_csv_path)
if 'Value' not in df.columns:
    raise ValueError("❌ CSVに 'Value' 列が見つかりません。")
timestamps = df['Value'].values  # NumPy配列に変換


# === フレーム数・サイズ・FPS ===
fps = 10.24  # 固定FPS
frame_count = int(min(cap1.get(cv2.CAP_PROP_FRAME_COUNT), cap2.get(cv2.CAP_PROP_FRAME_COUNT)))

# 動画1のサイズ
w1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
h1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"動画1のサイズ: {w1}x{h1}")

# 動画2のサイズ → 1.225倍に拡大（intで丸め）
w2_orig = int(cap2.get(cv2.CAP_PROP_FRAME_WIDTH))
h2_orig = int(cap2.get(cv2.CAP_PROP_FRAME_HEIGHT))
w2 = int(w2_orig * 1.225)
h2 = int(h2_orig * 1.225)
print(f"動画2のサイズ: {w2_orig}x{h2_orig} → 拡大後: {w2}x{h2}")

# === 配置位置の計算 ===
x1 = (canvas_width - w1) // 2  # video1のx位置（中央揃え）
# x1 = 400  # video1のx位置（左端から400）
x2 = x1 + w1 + 50  # video2のx位置（video1との間隔50）

y1 = (canvas_height - h1) // 2  # video1のy位置（中央揃え）
y2 = (canvas_height - h2) // 2  # video2のy位置（中央揃え）
print(f"配置位置: video1({x1}, {y1}), video2({x2}, {y2})")

white = (255, 255, 255)
border_thickness = 3

# === 出力設定 ===
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter(output_path, fourcc, fps, (canvas_width, canvas_height))

# === 結合処理 ===
for i in tqdm(range(frame_count)):
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    if not ret1:
        frame1 = np.zeros((h1, w1, 3), dtype=np.uint8)
    if not ret2:
        frame2 = np.zeros((h2, w2, 3), dtype=np.uint8)

    # video2を拡大
    frame2_resized = cv2.resize(frame2, (w2, h2), interpolation=cv2.INTER_LINEAR)

    # 黒背景生成
    canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)

    # video1を貼り付け
    canvas[y1:y1+h1, x1:x1+w1] = frame1

    # video2を貼り付け
    canvas[y2:y2+h2, x2:x2+w2] = frame2_resized

    # 枠線
    # cv2.rectangle(canvas, (x1, y1), (x1 + w1, y1 + int(h1 *0.83) ), white, border_thickness)
    # cv2.rectangle(canvas, (x1, y1), (x1 + int(w1 * 0.836), y1 + h1), white, border_thickness)
    cv2.rectangle(canvas, (x1, y1), (x1 + w1, y1 + int(h1*0.85) ), white, border_thickness)
    cv2.rectangle(canvas, (x1, y1), (x1 + int(w1*0.87), y1 + h1), white, border_thickness)

    # タイムスタンプ
    canvas = put_text_arial(canvas, f"T={timestamps[i]:.3f} s", (x1 + 20, y1 + 20), white)

    # 書き込み
    out.write(canvas)

# === 後始末 ===
cap1.release()
cap2.release()
out.release()
print("✅ 保存完了:", output_path)


動画1のサイズ: 945x828
動画2のサイズ: 273x286 → 拡大後: 334x350
配置位置: video1(677, 36), video2(1672, 275)


  0%|          | 0/665 [00:00<?, ?it/s]

100%|██████████| 665/665 [00:20<00:00, 32.23it/s]

✅ 保存完了: G:\kumamushi\Kumamushi_color\Kumamushi_color\merged_output_newlayout_outline_centered.avi


In [13]:
import cv2
import numpy as np
import os
from PIL import ImageFont, ImageDraw, Image

# === Arial Italic フォントのパス（Windowsの場合）===
font_path = "C:/Windows/Fonts/ariali.ttf"
font_size = 40
font = ImageFont.truetype(font_path, font_size)

def put_text_arial_italic(img, text, position, color=(255, 255, 255)):
    # OpenCVのBGR画像をPILのRGBに変換
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    # PIL画像をOpenCVのBGRに戻す
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)


# 入出力ファイルパス
base_path = "G:\kumamushi\Kumamushi_color\Kumamushi_color"
input_path = os.path.join(base_path, "merged_output_newlayout_outline_centered.avi")
output_path = os.path.join(base_path, "merged_output_newlayout_centered.mp4")

# 動画読み込み
cap = cv2.VideoCapture(input_path)
if not cap.isOpened():
    raise RuntimeError("❌ 入力動画が開けません")

# 基本情報取得
fps = 10.24  # 固定FPS
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 出力設定（同じサイズ・fpsで新動画を保存）
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
if not out.isOpened():
    raise RuntimeError("❌ 出力動画が作成できません")

white = (255, 255, 255)

v_shift = 60
h_shift = 10

# # # 白線描画座標（必要ならここで調整してください）
# v_start = (x1 + int(w1 * 0.836), y1)
# v_end = (v_start[0] + 5, v_start[1] + h1)
# h_start = (x1, y1 + int(h1 *0.83))
# h_end = (h_start[0] + w1, h_start[1] + 5)

# スケールバーの座標（必要ならここで調整してください）
scalebar_start = (x1 + int(w1 * 0.87) - 150 , y1 + int(h1 *0.86) - v_shift + 20) 
scalebar_end = (scalebar_start[0] + 126, scalebar_start[1] + 6) 
scalebar_thickness = -1

print(f"▶ 白線とラベル描画中: {frame_count} フレーム")

# ラベル座標とテキスト
label_positions = {
    "xy": (x1 + 10, y1 + int(h1 *0.86) - v_shift),
    "xz": (x1 + 10, y1 +int(h1 *0.83) + 10),
    "zy": (x1 + w1 - 50, y1 + int(h1 *0.86) - v_shift),
}

thickness = -1  # 塗りつぶし（白線用）

for i in range(frame_count):
    ret, frame = cap.read()
    if not ret:
        break

    # # 縦線：白塗り矩形
    # cv2.rectangle(frame, v_start, v_end, white, thickness=thickness)
    # # 横線：白塗り矩形
    # cv2.rectangle(frame, h_start, h_end, white, thickness=thickness)
    # スケールバー：白塗り矩形
    cv2.rectangle(frame, scalebar_start, scalebar_end, white , thickness=scalebar_thickness)

    # ラベル描画（Arial Italicフォント使用）
    for text, position in label_positions.items():
        frame = put_text_arial_italic(frame, text, position, white)

    # フレーム書き出し
    out.write(frame)

# 後処理
cap.release()
out.release()
print("✅ 完了:", output_path)


▶ 白線とラベル描画中: 665 フレーム
✅ 完了: G:\kumamushi\Kumamushi_color\Kumamushi_color\merged_output_newlayout_centered.mp4
